In [1]:
import numpy as np
import torch

In [2]:
image = torch.zeros((5, 3, 1024, 1024))

In [3]:
from GNN.src.dnn_f import DNN_F
model = DNN_F()
model

In [5]:
model(image).shape

torch.Size([5])

In [11]:
!ls "../Data/FinalSplits/"

Test_Both.pkl             Test_VEGF.pkl             Train_VEGF_normalized.pkl
Test_Both_normalized.pkl  Test_VEGF_normalized.pkl  Valid_Both_normalized.pkl
Test_TER.pkl              Train_Both_normalized.pkl Valid_TER_normalized.pkl
Test_TER_normalized.pkl   Train_TER_normalized.pkl  Valid_VEGF_normalized.pkl


In [20]:
import argparse
from tqdm import tqdm

import math
import os
from pathlib import Path

import pandas as pd
import numpy as np
import pickle
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

def check_for_nan(dataset):
    for i, data in enumerate(dataset):
        if torch.isnan(data.x).any():
            print(f"NaN found in features at index {i}")
        if torch.isnan(data.y).any():
            print(f"NaN found in target at index {i}")

def load_dataset_from_pickle(pickle_file):
    with open(pickle_file, 'rb') as f:
        dataset = pickle.load(f)
    if isinstance(dataset, list) and all(isinstance(d, Data) for d in dataset):
        return dataset
    else:
        raise ValueError("The loaded dataset is not a list of Data objects).")

def get_loaders(data_dirs, target, batch_size, print_data_stats = True, print_detailed = False):
    train_pickle_file = data_dirs[f"Train_{target}"]
    val_pickle_file = data_dirs[f"Valid_{target}"]
    test_pickle_file = data_dirs[f"Test_{target}"]

    # train_dataset = load_dataset_from_pickle(train_pickle_file)
    # val_dataset = load_dataset_from_pickle(val_pickle_file)
    test_dataset = load_dataset_from_pickle(test_pickle_file)

    # check_for_nan(train_dataset)
    # check_for_nan(val_dataset)
    check_for_nan(test_dataset)

    num_features = test_dataset[0].x.shape[1]  # Number of features per node
    num_targets = test_dataset[0].y.shape[0]

    detail_list = [num_features, num_targets]

    # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return test_loader, detail_list

In [21]:
data="../Data/FinalSplits"
norm_string=""
target="TER"
data_dirs = {}
for data_type in ['TER', 'VEGF', 'Both']:
    data_dirs[f"Train_{data_type}"] = f"{data}/Train_{data_type}{norm_string}.pkl"
    data_dirs[f"Valid_{data_type}"] = f"{data}/Valid_{data_type}{norm_string}.pkl"
    data_dirs[f"Test_{data_type}"] = f"{data}/Test_{data_type}{norm_string}.pkl"

test_loader, data_details = get_loaders(data_dirs, target, 64)

In [23]:
import GNN.src.gnn_multiple as GCNs
model_constructors = {
    "G2_D2": GCNs.GCN_G2_D2,
    "G2_D3": GCNs.GCN_G2_D3,
    "G2_D4": GCNs.GCN_G2_D4,
    "G2_D5": GCNs.GCN_G2_D5,
    "G3_D2": GCNs.GCN_G3_D2,
    "G3_D3": GCNs.GCN_G3_D3,
    "G3_D4": GCNs.GCN_G3_D4,
    "G3_D5": GCNs.GCN_G3_D5,
    "G4_D2": GCNs.GCN_G4_D2,
    "G4_D3": GCNs.GCN_G4_D3,
    "G4_D4": GCNs.GCN_G4_D4,
    "G4_D5": GCNs.GCN_G4_D5,
    "G5_D2": GCNs.GCN_G5_D2,
    "G5_D3": GCNs.GCN_G5_D3,
    "G5_D4": GCNs.GCN_G5_D4,
    "G5_D5": GCNs.GCN_G5_D5
}

model_class = model_constructors["G3_D2"]
standard_model = model_class(*data_details)
standard_model

GCN_G3_D2(
  (conv1): GATv2Conv(77, 128, heads=8)
  (norm1): BatchNorm(1024)
  (conv2): GATv2Conv(1024, 128, heads=8)
  (norm2): BatchNorm(1024)
  (conv3): GATv2Conv(1024, 1, heads=1)
  (norm3): BatchNorm(1)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear1): Linear(in_features=1, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=1, bias=True)
)

In [25]:
from GNN.src.gnn_modular import Modular_GCN
modular_gcn = Modular_GCN(*data_details)
modular_gcn

Modular_GCN(
  (gats): ModuleList(
    (0): GATv2Conv(77, 128, heads=8)
    (1): GATv2Conv(1024, 128, heads=8)
    (2): GATv2Conv(1024, 1, heads=1)
  )
  (b_norms): ModuleList(
    (0-1): 2 x BatchNorm(1024)
    (2): BatchNorm(1)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (dense_head): ModuleList(
    (0): Linear(in_features=1, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=1, bias=True)
  )
)